# Ronin Tax Reporter #
This notebook will produce a CSV file compatible with Koinly, which can be modified for other tax report formats. The resulting file may possibly contain errors or duplications, so please check the file manually before using it.

In [ ]:
import requests
import pandas as pd
from datetime import datetime as dt

In [ ]:
#address = 'ACCOUNT_ADDRESS'
address = input("Enter your Ronin address:")

In [ ]:
if address[:6] == 'ronin:':
    address = address[6:]
if address[:2] == '0x':
    address = address[2:]

In [ ]:
if len(address) != 40:
    print("This is not a valid Ronin address")

In [ ]:
tokens = {'axie': 'AXIE #', 'land': 'LAND #', 'item': 'ITEM #', 'rune': 'RUNE #', 'charm': 'CHARM #', 'slp': 'SLP', 'axs': 'AXS', 'usdc-weth-lp': 'USDC-WETH LP',
                                'ron-weth-lp': 'RON-WETH LP', 'axs-weth-lp': 'AXS-WETH LP', 'slp-weth-lp': 'SLP-WETH LP', 'usdc': 'USDC', 'aec': 'AEC', 'weth': 'ETH'}

In [ ]:
multipliers = {'slp': 1, 'axs': 1e-18, 'usdc-weth-lp': 1e-18, 'ron-weth-lp': 1e-18, 'axs-weth-lp': 1e-18,
               'slp-weth-lp': 1e-18, 'usdc': 1e-6, 'aec': 1, 'weth': 1e-18}

In [ ]:
report = pd.DataFrame(columns = ['Date', 'Sent Amount', 'Sent Currency', 'Received Amount', 'Received Currency',
                      'Fee Amount', 'Fee Currency', 'Description', 'TxHash'])

In [ ]:
for token in list(tokens.keys())[:5]:
    page = 1
    print(tokens[token][:-2] + "S: Completed pages: ", end="")
    while True:
        url = 'https://ronin.rest/archive/getWalletTransferHistory/' + token + '/ronin:b7e5e61b89cdd425a729b0b76bef5c77986cb475?page=' + str(page)
        response = requests.request("GET", url, headers={}, data={})
        if len(response.json()["transfers"]) == 0:
            print("")
            break
        for item in response.json()["transfers"]:
            txId = item['transaction_id']
            url = 'https://ronin.rest/ronin/getTransactionReceipt/' + txId
            receipt = requests.request("GET", url, headers={}, data={})
            url = 'https://ronin.rest/ronin/decodeTransactionReceipt/' + txId
            decode = requests.request("GET", url, headers={}, data={})
            blockHt = receipt.json()['blockNumber']
            url = 'https://ronin.rest/ronin/getBlock/' + str(blockHt)
            block = requests.request("GET", url, headers={}, data={})
            date = dt.fromtimestamp(block.json()['timestamp'])
            date = dt.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            fee = 0
            if txId not in report.TxHash:
                fee = receipt.json()['gasUsed'] * receipt.json()['effectiveGasPrice']
            desc = tokens[token] + str(int('0x' + item['value_or_token_id'], base=16))
            if item['from'] == '0x' + address:
                report = report.append({'Date': date, 'Sent Amount': 1.0, 'Sent Currency': 'NULL', 'Fee Amount': fee * 1e-18,
                      'Fee Currency': 'RON', 'Description': desc, 'TxHash': txId}, ignore_index=True)
            else:
                report = report.append({'Date': date, 'Received Amount': 1.0, 'Received Currency': 'NULL', 'Fee Amount': fee * 1e-18,
                      'Fee Currency': 'RON', 'Description': desc, 'TxHash': txId}, ignore_index=True)
        print(page, end=" ")
        page += 1

In [ ]:
for token in list(tokens.keys())[5:]:
    page = 1
    print(tokens[token] + ": Completed pages: ", end="")
    while True:
        url = 'https://ronin.rest/archive/getWalletTransferHistory/' + token + '/ronin:b7e5e61b89cdd425a729b0b76bef5c77986cb475?page=' + str(page)
        response = requests.request("GET", url, headers={}, data={})
        if len(response.json()["transfers"]) == 0:
            print("")
            break
        for item in response.json()["transfers"]:
            txId = item['transaction_id']
            url = 'https://ronin.rest/ronin/getTransactionReceipt/' + txId
            receipt = requests.request("GET", url, headers={}, data={})
            url = 'https://ronin.rest/ronin/decodeTransactionReceipt/' + txId
            decode = requests.request("GET", url, headers={}, data={})
            blockHt = receipt.json()['blockNumber']
            url = 'https://ronin.rest/ronin/getBlock/' + str(blockHt)
            block = requests.request("GET", url, headers={}, data={})
            date = dt.fromtimestamp(block.json()['timestamp'])
            date = dt.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            fee = 0
            if txId not in report.TxHash:
                fee = receipt.json()['gasUsed'] * receipt.json()['effectiveGasPrice']
            amount = int('0x' + item['value_or_token_id'], base=16) * multipliers[token]
            currency = tokens[token]
            if item['from'] == '0x' + address:
                report = report.append({'Date': date, 'Sent Amount': amount, 'Sent Currency': currency,
                      'Fee Amount': fee * 1e-18, 'Fee Currency': 'RON', 'TxHash': txId}, ignore_index=True)
            else:
                report = report.append({'Date': date, 'Received Amount': amount, 'Received Currency': currency,
                      'Fee Amount': fee * 1e-18, 'Fee Currency': 'RON', 'TxHash': txId}, ignore_index=True)
            if token == 'weth':
                for entry in decode.json()['decodedInput']:
                    if 'address' in entry:
                        if entry['address'].lower() == '0xe514d9deb7966c8be0ca922de8a064264ea6bcd4':
                            for entry1 in decode.json()['decodedInput']:
                                if 'name' in entry1:
                                    if entry1['name'] == 'Swap':
                                        if entry1['events'][2]['value'] != '0':
                                            ronAmt = int(entry1['events'][2]['value'])
                                            report = report.append({'Date': date, 'Sent Amount': ronAmt * 1e-18,
                                                                    'Sent Currency': 'RON', 'TxHash': txId}, ignore_index=True)
                                            break
                                        if entry1['events'][4]['value'] != '0':
                                            ronAmt = int(entry1['events'][4]['value'])
                                            report = report.append({'Date': date, 'Received Amount': ronAmt * 1e-18,
                                                                    'Received Currency': 'RON', 'TxHash': txId}, ignore_index=True)
                                            break                            
                                    if entry1['name'] == 'Mint':
                                        ronAmt = int(entry1['events'][2]['value'])
                                        report = report.append({'Date': date, 'Sent Amount': ronAmt * 1e-18,
                                                                'Sent Currency': 'RON', 'TxHash': txId}, ignore_index=True)
                                        break
                                    if entry1['name'] == 'Burn':
                                        ronAmt = int(entry1['events'][2]['value'])
                                        report = report.append({'Date': date, 'Received Amount': ronAmt * 1e-18,
                                                                'Received Currency': 'RON', 'TxHash': txId}, ignore_index=True)
                                        break
                            break             
        print(page, end=" ")
        page += 1

In [ ]:
report

In [ ]:
filename = "Ronin_txs_" + address + "_" + dt.strftime(dt.now(), "%Y%m%d_%H%M%S") + ".csv"

In [ ]:
report.to_csv(filename, index=False)